In [244]:
# Import libraries
import requests
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import os, os.path as osp
import re

In [335]:
DATA_FOLDER = 'data'
SAMPLE_DATA_FOLDER = DATA_FOLDER + '/htmlSample/'

#SAMPLE_DATA_FOLDER='htmlSample/'
SAMPLE_FILE='data/htmlSample/ff6da2b8d426c56ae77beda595bdcfea.html' #Recipe
SAMPLE_FILE_2='data/htmlSample/ff727d984c9c0048def173c4c97ab52e.html'
SAMPLE_FILE_3='data/htmlSample/ff6f004f00135caa7d6f91b27cbd3a4a.html'
SAMPLE_FILE_4='data/htmlSample/ff70a9eb4e8de4c2f72f281993ab7a55.html'
SAMPLE_FILE_5='data/htmlSample/ff7208a2993418a1b6d50c88b279f229.html'
SAMPLE_FILE_6='data/htmlSample/ff6f69af86e33565848f2c7366a793d7.html'

In [424]:
meat_products = ['meat', 'beef', 'chicken', 'fish', 'vanilla','basil','lamb','Lamb','flour','walnuts']
units = ['pound','gram','oz','ounce','kg','kilogram' ]

In [404]:
#code to run for each html file
with open(SAMPLE_FILE_6) as f:
    page = f.read()
    soup = BeautifulSoup(page, 'html.parser')
    ing_wrap_1=soup.find_all('li', class_="ingredient")
    ing_amnts=[]
    ings=[]
    for i in ing_wrap_1:
                ing_amnts.append((i.find('span',class_='value').text+ ' '+i.find('span',class_='type').text))
                ings.append(i.find('span', class_='name').text)
                #print(i)
                print('-----')
    #print(ing_wrap_1)
    print(ing_amnts)
    print(ings)

-----
-----
-----
-----
-----
-----
-----
['1 1/2 teaspoons', '1/2 teaspoon', '1/4 teaspoon', '1  teaspoon', '4 (4  ounce)', '2  tablespoons', '2  tablespoons']
['\ndried thyme\n', '\nsalt\n', '\nblack pepper\n', '\nolive oil\n', '\nboneless skinless chicken breast halves\n', '\n\t\t\t\t\t\t\t \n\t\t\t\t\t\t\tbalsamic vinegar\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\nhoney\n']


In [411]:
#code to run for each html file
with open(SAMPLE_FILE_6) as f:
    page = f.read()
    soup = BeautifulSoup(page, 'html.parser')
    
    ingredient_wrapper, is_recipe, ingredients, ingredient_amounts=analyse_recipe(soup,page)
    print(ingredients)

['\ndried thyme\n', '\nsalt\n', '\nblack pepper\n', '\nolive oil\n', '\nboneless skinless chicken breast halves\n', '\n\t\t\t\t\t\t\t \n\t\t\t\t\t\t\tbalsamic vinegar\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\nhoney\n']


In [421]:
ingredient_amounts[ingredients.index(ingredients[4])]

'4 (4  ounce)'

In [326]:
ingredients.append('3 pounds beef') #added on beef to test
ingredients

['\n                    8 cups all-purpose flour',
 '\n                    1 1/2 teaspoons salt',
 '\n                    2 cups shortening',
 '\n                    4 eggs, beaten',
 '\n                    1 (12 fluid ounce) can evaporated milk',
 '\n                    1 (0.6 ounce) cake compressed fresh yeast',
 '\n                    1/4 cup white sugar',
 '\n                    \xa0',
 '\n                    8 cups chopped walnuts',
 '\n                    2 cups white sugar',
 '\n                    2 teaspoons ground cinnamon',
 '\n                    1/2 cup margarine',
 '\n                    1 egg, beaten',
 '\n                    1/3 cup milk',
 '\n                    1 cup white sugar for decoration',
 '3 pounds beef']

In [425]:
def extract_amount(ings, ing_amnt):
    """
    takes as argument the ingredients and ingredient amounts (if available)
    returns a list of all meat ingredients in the recipe, the amount of the ingredient in kg and a boolean contains_meat
    """

    meat_ingredients_full = []
    meat_ingredients_base = []
    ing_quant=[]
    
    contains_meat=False
    #Find meat ingredients
    for i in ings:
        for meat_product in meat_products:
            if i != None:
                if meat_product in i:
                    contains_meat=True
                    meat_ingredients_full.append(i)
                    meat_ingredients_base.append(meat_product)

    print(meat_ingredients_base)
    print(meat_ingredients_full)
    #extract amount if amount is not directly available
    if not ing_amnt: #if amount is empty
        for meat_i in meat_ingredients_full:
            meat_i_quant_kg=0
            print('------------')
            print('Ingredient: ', meat_i)
            
            print('Extracting amount:')
            #amount=re.match(r'\d', meat_i).group()
            amount=re.search(r'\d', meat_i).group()

            print('quantity=',amount)
            
            print('Extracting units:')
            for u in units: 
                if u in meat_i:
                    print('Quantity in '+ u + ' converted to ')
                    meat_i_quant_kg = convert_to_kg(float(amount),u)
            ing_quant.append(meat_i_quant_kg)
            if meat_i_quant_kg==0:
                print('Units not recognized for: '+meat_i)
                
                
    else:# if amount is directly available through the soup
        print('Amount available')
        for meat_i in meat_ingredients_full:
            meat_i_quant_kg=0
            print('------------')
            print('Ingredient: ', meat_i)
            #get index of ingredient
            amount=ing_amnt[ings.index(meat_i)]
            print('Extracting amount:')
            print(amount)
            amount_val=re.search(r'\d', amount).group()
            print('quantity=',amount_val)
            print('Extracting units:')
            for u in units: 
                if u in amount:
                    print('Quantity in '+ u + ' converted to ')
                    meat_i_quant_kg = convert_to_kg(float(amount_val),u)
            ing_quant.append(meat_i_quant_kg)
            if meat_i_quant_kg==0:
                print('Units not recognized for: '+amount+meat_i)
    return meat_ingredients_full, meat_ingredients_base, ing_quant,contains_meat


    
extract_amount(ingredients,ingredient_amounts)


['chicken']
['\nboneless skinless chicken breast halves\n']
Amount available
------------
Ingredient:  
boneless skinless chicken breast halves

Extracting amount:
4 (4  ounce)
quantity= 4
Extracting units:
Quantity in ounce converted to 
0.11339797017633384 kg


(['\nboneless skinless chicken breast halves\n'],
 ['chicken'],
 [0.11339797017633384],
 True)

In [332]:
measurements = ['pound','pounds','grams','gram','oz','kg','kilogram']

def convert_to_kg(quant, unit):
    if (unit=='kilogram') or (unit=='kg'):
        amnt_kg=quant
        print(quant,'kg')
    elif (unit=='pound') or (unit=='lb') or (unit=='lbs') or (unit=='pounds'):
        amnt_kg=quant/2.205
        print(amnt_kg,'kg')
    elif(unit=='g') or (unit=='gram') or (unit =='grams'):
        amnt_kg=quant/1000
        print(amnt_kg,'kg')     
    elif(unit=='oz') or (unit=='ounce'):
        amnt_kg=quant/35.274
        print(amnt_kg, 'kg')
    return(amnt_kg)
    
convert_to_kg(1,'oz')

0.02834949254408346 kg


0.02834949254408346

In [405]:
def analyse_recipe(soup, page):
    """
    takes soup, page as argument
    returns ingredientwrapper, s=spacer
    """
    ings = []
    ing_amnts = []
    if 'Allrecipes' in soup.title.string:
        print('This recipe is from Allrecipes')
        is_recipe = True
        ing_wrap=soup.find_all(itemprop="recipeIngredient")
        if not ing_wrap:
            print('alternative format used')
            ing_wrap=soup.find_all('li', class_="plaincharacterwrap ingredient")
            for ing in ing_wrap:
                ings.append(ing.text)

        else:
            for ing in ing_wrap:
                ing_amnts.append(ing.find(itemprop='amount').text)
                ings.append(ing.find(itemprop='name').text)
              
    elif 'Epicurious' in soup.title.string:
        print('This recipe is from Epicurious')
        ing_wrap=soup.find('div', id="ingredients")
        for ing in ing_wrap:
            ings.append(ing.string)

    elif 'Food.com' in soup.title.string:
        ing_wrap=soup.find_all('li', class_="ingredient")
        is_recipe = True
        for ing in ing_wrap:
            ing_amnts.append((ing.find('span',class_='value').text+ ' '+ing.find('span',class_='type').text))
            ings.append(ing.find('span', class_='name').text)
            
    elif 'MyRecipes' in soup.title.string:
        print('This recipe is from MyRecipes')
        ing_wrap=soup.find_all(itemprop="ingredient")
        is_recipe = True
        for ing in ing_wrap:
            ings.append(ing.getText())
    # this really has to be analyzed more closely!!
    else:
        is_recipe = False
        ing_wrap=None

    return ing_wrap, is_recipe, ings, ing_amnts



In [215]:
#NEED TO MODIFY LOOP BELOW

1


In [334]:
#Nadine's loop for all recipes in folder
#PLAYGROUND WITH THE SAME LOOP, BUT PRINTS AND STUFF FOR DEBUGGING AND EXPERIMENTING
# data has following row structure
# RecipeName as Identifier - bool contains_meat - list of co2 ingredients - carbonFootprint - ingredients
data=[]
for filename in os.listdir(SAMPLE_DATA_FOLDER):
    with open(SAMPLE_DATA_FOLDER+filename) as f:
        isTrue=False
        count_exceptions=0
        print('------------------------------------')
        
        # introduce try/catch such that it does no longer stop, when not recoginzeing letter
        try:
            page = f.read()
            soup = BeautifulSoup(page, 'html.parser')
            
            ingredient_wrappers, is_recipe, ingredients, ingredient_amounts = analyse_recipe(soup,page)
            print('is recipe: ',is_recipe)
            print(ingredients)
            print(ingredient_amounts)
            #different tag for ingredient and different separators in ingredient list between the webpages
            if is_recipe:
                print('enter')
                #print(ingredient_wrappers)

                #meatlist, contains_meat = analyse_meat(ingredient_wrappers,s)
                #ingredients = get_ingredients(ingredient_wrappers)
                ingredients=extract_amount(ingredients, ingredient_amounts)

                print('Title: ', soup.title.string)
                print('filename: ', filename)
                print(ingredients)
                #data.append
                #add row to dataset only if recipe contains meat
                #data.append([soup.title.string, contains_meat, meatlist, carbon_fp(meatlist), ingredients])
            else:
                print(soup.title.string)
        except:
            print('exception')
            count_exceptions=count_exceptions+1
#print(data)
# save data as csv 

------------------------------------
This recipe is from MyRecipes
is recipe:  True
['\n \n Cooking spray\n \n', '\n3 cups\n sifted cake flour\n \n', '\n2 tablespoons\n unsweetened cocoa\n \n', '\n1 teaspoon\n baking soda\n \n', '\n1 teaspoon\n baking powder\n \n', '\n1/2 teaspoon\n salt\n \n', '\n1 2/3 cups\n granulated sugar\n \n', '\n1/2 cup\n butter, softened\n \n', '\n4 \n large egg whites\n \n', '\n2 cups\n fat-free buttermilk\n \n', '\n1 \n (1-ounce) bottle red food coloring\n \n', '\n1 teaspoon\n vanilla extract\n \n', '\n7 ounces\n 1/3-less-fat cream cheese\n \n', '\n1 teaspoon\n vanilla extract\n \n', '\n2 3/4 cups\n powdered sugar\n \n', '\n1/2 cup\n seedless raspberry jam\n \n']
[]
enter
['flour', 'vanilla', 'vanilla']
['\n3 cups\n sifted cake flour\n \n', '\n1 teaspoon\n vanilla extract\n \n', '\n1 teaspoon\n vanilla extract\n \n']
------------
Ingredient:  
3 cups
 sifted cake flour
 

Extracting amount:
quantity= 3
Extracting units:
Units not recognized for: 
3 cups
 sif

In [ ]:
# calculate carbon footprint
#input ingredients
#identify ingredient ammount
#output carbon footprint
def carbon_fp (l):
    """
    takes a list of ingredients contributing to co2 and returns carbon footprint
    """
    c=len(l)
    return c

